## Querying TF Serving through the REST API

In [1]:
import numpy as np
import tensorflow as tf
from tensorflow import keras

In [2]:
#reading an image filepath and preprocessing it 
def preprocess(filepath):
    img = tf.io.read_file(filepath)
    img = tf.io.decode_jpeg(img,channels=3)
    img_res = tf.image.resize(img,[200,200])
    img_exp = tf.expand_dims(img_res,0)
    return img_exp

In [3]:
img_path = 'images.jpeg'
X_test = preprocess(img_path)
X_lst = X_test.numpy().tolist()

In [4]:
import json 

input_data_json = json.dumps({
    "signature_name":"serving_default",
    "instances":X_lst
})

In [5]:
import requests

SERVER_URL = "http://localhost:8501/v1/models/age_detector_model:predict"
response = requests.post(SERVER_URL,data=input_data_json)
response.raise_for_status() #raise an exception incase of an error
response = response.json()

In [6]:
y_proba = np.array(response["predictions"])
y_proba.round(2)

array([[0.  , 0.  , 0.  , 0.  , 0.  , 0.01, 0.  , 0.98]])

## Querying TF Serving through the gRPC API


In [8]:
#creating the request
from tensorflow_serving.apis.predict_pb2 import PredictRequest

request = PredictRequest()
request.model_spec.name = model_name
request.model_spec.signature_name = 'serving_default'
request.inputs['conv2d_input'].CopyFrom(tf.make_tensor_proto(X_test))

In [9]:
#sending request to the server
import grpc
from tensorflow_serving.apis import prediction_service_pb2_grpc

channel = grpc.insecure_channel('localhost:8500')
predict_service = prediction_service_pb2_grpc.PredictionServiceStub(channel)
response = predict_service.Predict(request,timeout=10.0)

In [10]:
#convert the PredictResponse protocal buffer to a Tensor
output_name ='dense'
outputs_proto = response.outputs[output_name]
y_proba = tf.make_ndarray(outputs_proto)

In [11]:
y_proba.round(2)

array([[0.  , 0.  , 0.  , 0.  , 0.  , 0.01, 0.  , 0.98]], dtype=float32)